In [1]:
from reader import *

In [2]:
bunny_model = Reader.read_from_file('assets/bunny.obj')

In [3]:
bunny_model.vertices

[Vertex(x=-0.00341018, y=0.13031957, z=0.02175437),
 Vertex(x=-0.08171916, y=0.15250145, z=0.02965609),
 Vertex(x=-0.03054348, y=0.12477885, z=0.00109834),
 Vertex(x=-0.02490159, y=0.11211138, z=0.03756024),
 Vertex(x=-0.01840568, y=0.17843055, z=-0.02421958),
 Vertex(x=0.01906794, y=0.12144925, z=0.03196844),
 Vertex(x=0.0060412, y=0.12494359, z=0.03265289),
 Vertex(x=-0.01346903, y=0.16299355, z=-0.01200002),
 Vertex(x=-0.03439324, y=0.17236688, z=-0.00098213),
 Vertex(x=-0.08431416, y=0.10957263, z=0.00370973),
 Vertex(x=-0.04223354, y=0.17211574, z=-0.00417998),
 Vertex(x=-0.06330839, y=0.15660615, z=-0.01383879),
 Vertex(x=-0.07690395, y=0.16708033, z=-0.02693136),
 Vertex(x=-0.07225392, y=0.1153955, z=0.0516703),
 Vertex(x=0.01298133, y=0.11366375, z=0.03830295),
 Vertex(x=-0.03785728, y=0.17010102, z=0.0014236),
 Vertex(x=0.00486894, y=0.03796237, z=0.04586763),
 Vertex(x=-0.05718055, y=0.04091883, z=0.04630134),
 Vertex(x=-0.04520907, y=0.0388391, z=0.04450377),
 Vertex(x=-0.03

In [4]:
bunny_model.aabb.size()

Vertex(x=0.15515922000000001, y=0.15368612, z=0.12039380999999999)